In [17]:
import os, numpy as np, pandas as pd
from gurobipy import Model, GRB, quicksum

# Paths 
BASE_DIR = os.path.expanduser("~/Downloads/IEOR4004EProject1")
PATH_REG = os.path.join(BASE_DIR, "child_care_regulated.csv")
PATH_POP = os.path.join(BASE_DIR, "population.csv")
PATH_INC = os.path.join(BASE_DIR, "avg_individual_income.csv")
PATH_EMP = os.path.join(BASE_DIR, "employment_rate.csv")

# Policy / cost parameters (idealistic)
EXPANSION_COST_PER_SLOT = 250.0           # (legacy flat rate; kept for reference)
EQUIP_COST_0_5_PER_SLOT = 100.0           # equipment surcharge per 0–5 slot (new + expanded)
NEW_COST   = {"S": 65000.0, "M": 95000.0, "L": 115000.0}
NEW_TOTAL  = {"S": 100, "M": 200, "L": 400}
NEW_0_5_MAX = {"S": 50,  "M": 100, "L": 200}
NEW_5_12_MAX = {k: NEW_TOTAL[k] - NEW_0_5_MAX[k] for k in NEW_TOTAL}

# Facility level expansion rule
BASELINE_FIXED_IF_100PCT = 20000.0        # fixed part if expand >= 100% at a facility
BASELINE_PER_EXISTING    = 200.0          # per existing slot if expand >= 100%
MAX_EXP_FRACTION         = 1.20           # up to +120% of existing facility capacity
MAX_EXP_SLOTS_PER_FAC    = 500.0          # hard cap per facility
BIG_M                    = 500.0          # safe Big-M (matches MAX_EXP_SLOTS_PER_FAC)
EPS                      = 1e-6

# Load data
reg = pd.read_csv(PATH_REG)
pop = pd.read_csv(PATH_POP)
inc = pd.read_csv(PATH_INC)
emp = pd.read_csv(PATH_EMP)

# take only the first 5 digits of Zip
def _zip_trunc(val):
    s = ''.join(ch for ch in str(val) if ch.isdigit())
    return np.nan if not s else int(s[:5])

reg["zipcode"] = reg["zip_code"].apply(_zip_trunc)
pop["zipcode"] = pop["zipcode"].apply(_zip_trunc)
inc["zipcode"] = inc["ZIP code"].apply(_zip_trunc)          
emp["zipcode"] = emp["zipcode"].apply(_zip_trunc)

for d in (reg, pop, inc, emp):
    d.dropna(subset=["zipcode"], inplace=True)
    d["zipcode"] = d["zipcode"].astype(int)

# Aggregate existing capacity by Zip
# 0–5 existing = infant + toddler + preschool; older existing = school_age
reg["cap_0_5"]  = reg[["infant_capacity","toddler_capacity","preschool_capacity"]].sum(axis=1)
reg["cap_5_12"] = reg[["school_age_capacity"]].sum(axis=1)

by_zip = (reg.groupby("zipcode")
            .agg(cap0_5=("cap_0_5","sum"),
                 cap5_12=("cap_5_12","sum"),
                 total_cap=("total_capacity","sum"),
                 n_facilities=("facility_id","count"))
            .reset_index())

# Build ZIP table with socio economic & population info
df = (by_zip
      .merge(pop[["zipcode","-5","5-9","10-14"]], on="zipcode", how="left")
      .merge(inc[["zipcode","average income"]],   on="zipcode", how="left")
      .merge(emp[["zipcode","employment rate"]],  on="zipcode", how="left"))

df[["cap0_5","cap5_12","total_cap"]] = df[["cap0_5","cap5_12","total_cap"]].fillna(0.0)
df["n_facilities"] = df["n_facilities"].fillna(0).astype(int)

'''Non-overlapping population groups
0–5  := (-5) + 1/5*(5–9) (adds 5-year-olds)
6–9  := 4/5*(5–9) (ages 6–9)
10–12:= 3/5*(10–14) (ages 10–12)'''
df["pop_0_5"]   = df["-5"].fillna(0.0) + 0.2 * df["5-9"].fillna(0.0)
df["pop_6_9"]   = 0.8 * df["5-9"].fillna(0.0)
df["pop_10_12"] = 0.6 * df["10-14"].fillna(0.0)
df["pop_0_12"]  = df["pop_0_5"] + df["pop_6_9"] + df["pop_10_12"]

# High-demand classification
df["employment rate"]   = df["employment rate"].fillna(0.0)
df["average income"]    = df["average income"].fillna(1e9)
df["is_high_demand"]    = ((df["employment rate"] >= 0.60) | (df["average income"] <= 60000)).astype(int)

# Thresholds (strictly above desert line)
df["threshold_total"]   = np.where(df["is_high_demand"]==1, 0.50*df["pop_0_12"], (1/3.0)*df["pop_0_12"])
df["threshold_total"]   = np.ceil(df["threshold_total"] + 1.0)
df["threshold_0_5"]     = np.ceil((2.0/3.0)*df["pop_0_5"])

# facility list for expansion decisions
fac = reg[["facility_id","zipcode","total_capacity"]].copy()
fac = fac.dropna(subset=["facility_id","zipcode"]).copy()
fac["facility_id"] = fac["facility_id"].astype(str)
fac["zipcode"] = fac["zipcode"].astype(int)
fac["total_capacity"] = pd.to_numeric(fac["total_capacity"], errors="coerce").fillna(0.0)

# Zipcode to list of facility_ids
zip_to_facs = fac.groupby("zipcode")["facility_id"].apply(list).to_dict()
cap_by_fac  = fac.set_index("facility_id")["total_capacity"].to_dict()

'''per-facility tiered expansion rates 
Assumption:
tier S (<=100 existing slots): $100/slot
tier M (101..200): $150/slot
tier L (>200): $200/slot'''
EXP_COST_TIER = {"S": 100.0, "M": 150.0, "L": 200.0}

def _tier_from_C(C):
    if C <= 100.0:
        return "S"
    elif C <= 200.0:
        return "M"
    else:
        return "L"

exp_cost_fac = {f: EXP_COST_TIER[_tier_from_C(C)] for f, C in cap_by_fac.items()}  # per-facility $/slot

# compute deserts pre-intervention
df["existing_total"] = df["cap0_5"] + df["cap5_12"]
df["is_desert_pre"] = ((df["existing_total"] < df["threshold_total"]) |
                       (df["cap0_5"]       < df["threshold_0_5"])).astype(int)
desert_zips = set(df.loc[df["is_desert_pre"] == 1, "zipcode"].astype(int).tolist())

# Gurobi model
m = Model("IEOR4004E_Idealistic_Gurobi_facility_expansion_desert_only")

# New facility counts 
yS = {z: m.addVar(vtype=GRB.INTEGER, lb=0, name=f"yS[{z}]") for z in df["zipcode"]}
yM = {z: m.addVar(vtype=GRB.INTEGER, lb=0, name=f"yM[{z}]") for z in df["zipcode"]}
yL = {z: m.addVar(vtype=GRB.INTEGER, lb=0, name=f"yL[{z}]") for z in df["zipcode"]}

# new capacity allocations by age group 
u_new_0_5  = {z: m.addVar(vtype=GRB.CONTINUOUS, lb=0.0, name=f"u_new_0_5[{z}]")  for z in df["zipcode"]}
u_new_6_9  = {z: m.addVar(vtype=GRB.CONTINUOUS, lb=0.0, name=f"u_new_6_9[{z}]")  for z in df["zipcode"]}
u_new_10_12= {z: m.addVar(vtype=GRB.CONTINUOUS, lb=0.0, name=f"u_new_10_12[{z}]") for z in df["zipcode"]}

# facility level expansion variables by age group
eF_0_5   = {f: m.addVar(vtype=GRB.CONTINUOUS, lb=0.0, name=f"eF_0_5[{f}]")   for f in cap_by_fac}
eF_6_9   = {f: m.addVar(vtype=GRB.CONTINUOUS, lb=0.0, name=f"eF_6_9[{f}]")   for f in cap_by_fac}
eF_10_12 = {f: m.addVar(vtype=GRB.CONTINUOUS, lb=0.0, name=f"eF_10_12[{f}]") for f in cap_by_fac}
# expansion >= 100% at facility f?
yF_100p  = {f: m.addVar(vtype=GRB.BINARY, name=f"yF_100p[{f}]")               for f in cap_by_fac}

m.update()

def total_new(z):     return NEW_TOTAL["S"]*yS[z] + NEW_TOTAL["M"]*yM[z] + NEW_TOTAL["L"]*yL[z]
def max_new_0_5(z):   return NEW_0_5_MAX["S"]*yS[z] + NEW_0_5_MAX["M"]*yM[z] + NEW_0_5_MAX["L"]*yL[z]
def max_new_5_12(z):  return NEW_5_12_MAX["S"]*yS[z] + NEW_5_12_MAX["M"]*yM[z] + NEW_5_12_MAX["L"]*yL[z]

# Facility level expansion limits and trigger
for f, C_f in cap_by_fac.items():
    E_f = eF_0_5[f] + eF_6_9[f] + eF_10_12[f]
    m.addConstr(E_f <= MAX_EXP_FRACTION * C_f, name=f"exp_frac_cap[{f}]")
    m.addConstr(E_f <= MAX_EXP_SLOTS_PER_FAC,  name=f"exp_abs_cap[{f}]")
    # Trigger yF_100p = 1 iff E_f >= C_f
    m.addConstr(E_f - C_f * yF_100p[f] >= 0.0,                   name=f"trigger_lb[{f}]")
    m.addConstr(E_f <= (C_f - EPS) + BIG_M * yF_100p[f],         name=f"trigger_ub[{f}]")

# ZIP-level coverage and deserts-only investment 
dfi = df.set_index("zipcode")

for z, r in dfi.iterrows():
    facs = zip_to_facs.get(int(z), [])
    sum_e0_5   = quicksum(eF_0_5[f]   for f in facs)
    sum_e6_9   = quicksum(eF_6_9[f]   for f in facs)
    sum_e10_12 = quicksum(eF_10_12[f] for f in facs)

    total_after_0_5  = r["cap0_5"]  + sum_e0_5 + u_new_0_5[z]
    total_after_older= r["cap5_12"] + (sum_e6_9 + sum_e10_12) + (u_new_6_9[z] + u_new_10_12[z])
    total_after      = total_after_0_5 + total_after_older

    # Coverage constraints
    m.addConstr(total_after     >= r["threshold_total"], name=f"not_desert[{z}]")
    m.addConstr(total_after_0_5 >= r["threshold_0_5"],  name=f"u05_policy[{z}]")

    # New build allocation envelopes
    m.addConstr(u_new_0_5[z]                  <= max_new_0_5(z),  name=f"new_0_5_cap[{z}]")
    m.addConstr(u_new_6_9[z] + u_new_10_12[z] <= max_new_5_12(z), name=f"new_older_cap[{z}]")
    m.addConstr(u_new_0_5[z] + u_new_6_9[z] + u_new_10_12[z] == total_new(z), name=f"new_alloc_bal[{z}]")

    # Deserts-only investment
    if int(r["is_desert_pre"]) == 0:
        m.addConstr(yS[z] == 0, name=f"no_build_S_non_desert[{z}]")
        m.addConstr(yM[z] == 0, name=f"no_build_M_non_desert[{z}]")
        m.addConstr(yL[z] == 0, name=f"no_build_L_non_desert[{z}]")
        m.addConstr(u_new_0_5[z] == 0,   name=f"no_new_0_5_non_desert[{z}]")
        m.addConstr(u_new_6_9[z] == 0,   name=f"no_new_6_9_non_desert[{z}]")
        m.addConstr(u_new_10_12[z] == 0, name=f"no_new_10_12_non_desert[{z}]")
        for f in facs:
            m.addConstr(eF_0_5[f]   == 0, name=f"no_exp_0_5_non_desert[{f}]")
            m.addConstr(eF_6_9[f]   == 0, name=f"no_exp_6_9_non_desert[{f}]")
            m.addConstr(eF_10_12[f] == 0, name=f"no_exp_10_12_non_desert[{f}]")

# build + equipment(0–5) + per-slot expansion + baseline if >=100% 
build_cost  = quicksum(NEW_COST["S"]*yS[z] + NEW_COST["M"]*yM[z] + NEW_COST["L"]*yL[z] for z in dfi.index)
equip_cost  = quicksum(EQUIP_COST_0_5_PER_SLOT * (u_new_0_5[z]) for z in dfi.index) \
            + quicksum(EQUIP_COST_0_5_PER_SLOT * (eF_0_5[f])    for f in cap_by_fac)

# flat version, not used in this code but kept in case it's needed for debugging:
expand_slots = quicksum(eF_0_5[f] + eF_6_9[f] + eF_10_12[f] for f in cap_by_fac)

# tiered per-slot expansion cost by facility size
expand_cost = quicksum(
    exp_cost_fac[f] * (eF_0_5[f] + eF_6_9[f] + eF_10_12[f])
    for f in cap_by_fac
)

baseline_cost = quicksum(
    (BASELINE_FIXED_IF_100PCT + BASELINE_PER_EXISTING * cap_by_fac[f]) * yF_100p[f]
    for f in cap_by_fac
)

m.setObjective(build_cost + equip_cost + expand_cost + baseline_cost, GRB.MINIMIZE)

m.Params.OutputFlag = 1
m.optimize()

# Collect, save results
rows = []
for z, r in dfi.iterrows():
    y_s, y_m, y_l = int(round(yS[z].X)), int(round(yM[z].X)), int(round(yL[z].X))
    new_total = NEW_TOTAL["S"]*y_s + NEW_TOTAL["M"]*y_m + NEW_TOTAL["L"]*y_l

    facs = zip_to_facs.get(int(z), [])
    sum_e0_5   = sum(eF_0_5[f].X   for f in facs)
    sum_e6_9   = sum(eF_6_9[f].X   for f in facs)
    sum_e10_12 = sum(eF_10_12[f].X for f in facs)

    rows.append(dict(
        zipcode=int(z),
        is_high_demand=int(r["is_high_demand"]),
        is_desert_pre=int(r["is_desert_pre"]),
        pop_0_5=float(r["pop_0_5"]), pop_6_9=float(r["pop_6_9"]), pop_10_12=float(r["pop_10_12"]),
        pop_0_12=float(r["pop_0_12"]),
        threshold_total=float(r["threshold_total"]), threshold_0_5=float(r["threshold_0_5"]),
        existing_cap_total=float(r["existing_total"]),
        existing_cap_0_5=float(r["cap0_5"]),
        existing_cap_5_12=float(r["cap5_12"]),
        y_small=y_s, y_medium=y_m, y_large=y_l,
        u_new_0_5=float(u_new_0_5[z].X),
        u_new_6_9=float(u_new_6_9[z].X),
        u_new_10_12=float(u_new_10_12[z].X),
        e_0_5=float(sum_e0_5),
        e_6_9=float(sum_e6_9),
        e_10_12=float(sum_e10_12),
        new_total_slots=float(new_total)
    ))

# Facility-level audit
fac_rows = []
for f, C_f in cap_by_fac.items():
    zf = int(fac.loc[fac["facility_id"]==f, "zipcode"].iloc[0])
    fac_rows.append(dict(
        facility_id=f,
        zipcode=zf,
        existing_total=float(C_f),
        exp_rate_per_slot=float(exp_cost_fac[f]),  # UPDATED: report rate used
        exp_0_5=float(eF_0_5[f].X),
        exp_6_9=float(eF_6_9[f].X),
        exp_10_12=float(eF_10_12[f].X),
        expanded_slots=float(eF_0_5[f].X + eF_6_9[f].X + eF_10_12[f].X),
        triggered_baseline=int(round(yF_100p[f].X))
    ))

zip_df = pd.DataFrame(rows).sort_values("zipcode")
fac_df = pd.DataFrame(fac_rows).sort_values(["zipcode","facility_id"])

OUT1 = os.path.join(BASE_DIR, "idealistic_solution_by_zip.csv")
OUT2 = os.path.join(BASE_DIR, "idealistic_expansion_by_facility.csv")
zip_df.to_csv(OUT1, index=False)
fac_df.to_csv(OUT2, index=False)
print("\n=== Objective (USD) ===", f"{m.objVal:,.2f}")
print("Wrote:", OUT1)
print("Wrote:", OUT2)

Set parameter OutputFlag to value 1
Gurobi Optimizer version 12.0.2 build v12.0.2rc0 (mac64[arm] - Darwin 24.5.0 24F74)

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 72901 rows, 69544 columns and 307988 nonzeros
Model fingerprint: 0x4ea4fb75
Variable types: 50376 continuous, 19168 integer (15604 binary)
Coefficient statistics:
  Matrix range     [1e+00, 9e+02]
  Objective range  [1e+02, 2e+05]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e-06, 1e+04]
Presolve removed 72864 rows and 69491 columns
Presolve time: 1.14s
Presolved: 37 rows, 53 columns, 154 nonzeros
Variable types: 34 continuous, 19 integer (18 binary)
Found heuristic solution: objective 2.880101e+08
Found heuristic solution: objective 2.879843e+08

Root relaxation: objective 2.879243e+08, 19 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf |

In [5]:
# Step 1: Compute desert flag based on threshold
df["is_desert"] = (df["total_cap"] < df["threshold_total"]).astype(int)

# Step 2: Select and rename columns for clarity
summary = df[[
    "zipcode",
    "pop_0_5", "pop_0_12",
    "employment rate", "average income",
    "cap0_5", "total_cap",
    "is_high_demand",
    "threshold_total", "threshold_0_5",
    "is_desert"
]].copy()

summary.rename(columns={
    "pop_0_5": "children_0_5",
    "pop_0_12": "children_0_12",
    "cap0_5": "existing_slots_0_5",
    "total_cap": "existing_slots_total",
    "employment rate": "employment_rate",
    "average income": "average_income",
    "is_high_demand": "high_demand",
    "threshold_total": "required_total_slots",
    "threshold_0_5": "required_under5_slots",
    "is_desert": "is_desert_area"
}, inplace=True)

# Step 3: Optional — add summary statistics
num_deserts = summary["is_desert_area"].sum()
total_zipcodes = len(summary)
print(f"Total ZIP codes: {total_zipcodes}")
print(f"Childcare deserts: {num_deserts} ({num_deserts / total_zipcodes:.2%})")

# Step 4: Save CSV to Downloads folder
output_path = os.path.expanduser("~/Downloads/childcare_desert_summary.csv")
summary.to_csv(output_path, index=False)

print(f"\n✅ CSV file saved to: {output_path}")

Total ZIP codes: 1188
Childcare deserts: 888 (74.75%)

✅ CSV file saved to: /Users/windyzhou/Downloads/childcare_desert_summary.csv


In [15]:
summary_rows = []

for z, r in dfi.iterrows():
    # New builds
    y_s, y_m, y_l = int(round(yS[z].X)), int(round(yM[z].X)), int(round(yL[z].X))
    new_slots = (
        NEW_TOTAL["S"] * y_s +
        NEW_TOTAL["M"] * y_m +
        NEW_TOTAL["L"] * y_l
    )

    # Expansions at facilities in this ZIP
    facs = zip_to_facs.get(int(z), [])
    exp_0_5 = sum(eF_0_5[f].X for f in facs)
    exp_6_9 = sum(eF_6_9[f].X for f in facs)
    exp_10_12 = sum(eF_10_12[f].X for f in facs)
    exp_total = exp_0_5 + exp_6_9 + exp_10_12

    # Combined new + expanded
    total_added_0_5 = r["cap0_5"] + exp_0_5 + u_new_0_5[z].X
    total_added_all = r["total_cap"] + exp_total + new_slots

    summary_rows.append({
        "zipcode": int(z),
        "is_high_demand": int(r["is_high_demand"]),
        "existing_slots_total": float(r["total_cap"]),
        "existing_slots_0_5": float(r["cap0_5"]),
        "expanded_slots_total": float(exp_total),
        "expanded_slots_0_5": float(exp_0_5),
        "new_build_small": y_s,
        "new_build_medium": y_m,
        "new_build_large": y_l,
        "new_slots_total": float(new_slots),
        "total_added_slots_total": float(exp_total + new_slots),
        "total_added_slots_0_5": float(exp_0_5 + u_new_0_5[z].X),
        "final_total_slots": float(total_added_all),
        "threshold_total": float(r["threshold_total"]),
        "threshold_0_5": float(r["threshold_0_5"]),
    })

summary_df = pd.DataFrame(summary_rows).sort_values("zipcode")

# === Save to CSV ===
OUT_SUM = os.path.join(BASE_DIR, "idealistic_summary_by_ziplalal.csv")
summary_df.to_csv(OUT_SUM, index=False)

print(f"\n✅ Summary file saved to: {OUT_SUM}")
print(f"Total ZIPs: {len(summary_df)} | Avg new slots per ZIP: {summary_df['total_added_slots_total'].mean():.1f}")


✅ Summary file saved to: /Users/windyzhou/Downloads/IEOR4004EProject1/idealistic_summary_by_ziplalal.csv
Total ZIPs: 1188 | Avg new slots per ZIP: 807.6


In [11]:
zip_df["total_added"] = zip_df["new_total_slots"] + zip_df["e_0_5"] + zip_df["e_5_9"] + zip_df["e_9_12"]
zip_df["was_desert_before"] = (zip_df["existing_cap_total"] < zip_df["threshold_total"]).astype(int)
zip_df["was_desert_after"]  = (zip_df["existing_cap_total"] + zip_df["total_added"] < zip_df["threshold_total"]).astype(int)

print(zip_df.groupby("was_desert_before")[["total_added"]].mean())

KeyError: 'e_5_9'